<center><a href="https://rapids.ai/"><img src="img/rapids.png" alt="rapids_logo" style="width: 1500px;"/></a></center>

# <font color='#8735fb'> Data Preprocessing

In this Notebook, we are going to take the `airlines_raw_data_full.csv` data file we previously downloaded, and we are going to process it into a format that will be more suited to data visualization and model training.

The first thing we will want to do with this data is merge it with the `carriers.csv` and `airports.csv` data. Luckily, these data files all share a column, so a standard merge operation will be trivial.

After creating a single DataFrame, we will need to clean it up. Here are a few of the concerns we will be addressing:

* There are many columns that we may not be interested in
* There may be duplicate columns
* There may be rows with missing data points
* There may be "categorical data" that is not in an ML-friendly format
* Some of the column names may be hard to understand


In [1]:
# Define some global directory/file names that change based on the environment
import cudf

# Set data directory as configured in the workflow
data_dir = '/home/ec2-user/SageMaker/data/'

# Set the name for input files
data_file = data_dir + 'airlines_raw_data.csv'
data_file_full = data_dir + 'airlines_raw_data_full.csv'

# Set the name for output processed files
processed_data_file = data_dir + 'airlines_processed_data.csv'
processed_data_file_full = data_dir + 'airlines_processed_data_full.csv'

# Set the name for additional CSV file with additional airport/airline metadata
carriers_file = data_dir + 'carriers.csv'
airports_file = data_dir + 'airports.csv'

### <font color='#8735fb'> Ingest
We start by selecting a subset of the columns and reading only that data into a cuDF DataFrame.

We get this information by inspecting the column headers and using our subject matter expertise to determine which columns contain which data type.

When loading specific columns from a data file, RAPIDS requires we tell it the intended data type for each column.

In [2]:
# Load the column names and data types for the data based on documentation and subject matter expertise from external sources

# Set the columns and their datatypes for conversion and parsing
cols = ['Flight_Number_Reporting_Airline', 'Year', 'Quarter', 'Month', 'DayOfWeek', 'DOT_ID_Reporting_Airline', 'OriginCityMarketID',
        'DestCityMarketID', 'DepTime', 'DepDelay', 'DepDel15', 'ArrTime', 'ArrDelay', 'ArrDel15', 'CRSDepTime', 'CRSArrTime', 'AirTime',
        'Distance', 'Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'Origin', 'OriginCityName', 'Dest', 'DestCityName', 'Cancelled']

dtypes = {'Flight_Number_Reporting_Airline': 'float32', 'Year': 'float32', 'Quarter': 'float32', 'Month': 'float32', 'DayOfWeek': 'float32',
          'DOT_ID_Reporting_Airline': 'float32', 'OriginCityMarketID': 'float32', 'DestCityMarketID': 'float32', 'DepTime': 'float32',
          'DepDelay': 'float32', 'DepDel15': 'int', 'ArrTime': 'float32', 'ArrDelay': 'float32', 'ArrDel15': 'int', 'CRSDepTime': 'float32',
          'CRSArrTime': 'float32', 'AirTime': 'float32', 'Distance': 'float32', 'Reporting_Airline': 'str', 'IATA_CODE_Reporting_Airline': 'str',
          'Origin': 'str', 'OriginCityName': 'str', 'Dest': 'str', 'DestCityName': 'str', 'Cancelled': 'str'}

categorical_columns = ['Flight_Number_Reporting_Airline', 'DepTime', 'ArrTime', 'CRSDepTime', 'CRSArrTime', 'Reporting_Airline', 'Origin',
                       'OriginCityName', 'Dest', 'DestCityName', 'Airline']

In [3]:
# Ingest - Read the CSV files into the DataFrame
data = cudf.read_csv(data_file, cols=cols, dtypes=dtypes)[cols] # Read in data, ignoring any column not in cols
carriers = cudf.read_csv(carriers_file)
airports = cudf.read_csv(airports_file, usecols=['iata_code', 'latitude_deg', 'longitude_deg',  'elevation_ft'])         

### <font color='#8735fb'> Merge
We then merge all three data files using the common `iata_code` to correlate the airline carriers to the airport of operation with the cuDF [`merge`](https://docs.rapids.ai/api/cudf/stable/api.html?highlight=merge#cudf.core.dataframe.DataFrame.merge) function.

In [4]:
# Merge - Combine the external data with the airline data         
data = cudf.merge(data, carriers, left_on='IATA_CODE_Reporting_Airline', right_on='Code', how='left')
data = cudf.merge(data, airports, left_on='Dest', right_on='iata_code', how='left')
data = cudf.merge(data, airports, left_on='Origin', right_on='iata_code', how='left')

### <font color='#8735fb'> Rename

We make some of the columns more human readable by using the cuDF [`rename`](https://docs.rapids.ai/api/cudf/stable/api.html?highlight=rename#cudf.core.dataframe.DataFrame.rename) function.

In [5]:
# Rename - Add clarity to the combined dataset by renaming columns
data = data.rename(columns= { 'latitude_deg_x' : 'dest_lat', 'longitude_deg_x': 'dest_long',
                              'latitude_deg_y' : 'origin_lat', 'longitude_deg_y': 'origin_long',
                              'elevation_ft_x' : 'dest_elevation', 'elevation_ft_y' : 'origin_elevation',
                              'Description' : 'Airline'})

### <font color='#8735fb'> Cleanup

During the merge process, we noticed that `airlines_raw_data_full.csv` and `airports.csv` both contained the same data under different names. We can get rid of this unnecessary data to speed up our model training and reduce the DataFrame size using the [`drop`](https://docs.rapids.ai/api/cudf/stable/api.html?highlight=drop#cudf.core.dataframe.DataFrame.drop) function.

Similarly, the data contains several rows with missing values. This will likely break our model training algorithms later, so we drop these rows using the [`dropna`](https://docs.rapids.ai/api/cudf/stable/api.html?highlight=dropna#cudf.core.dataframe.DataFrame.dropna) function. Only a very small fraction of flights have missing data (about 0.021 percent in 2019), so we can be confident this will have minimal impact on our model.

In [6]:
# Remove duplicates columns
data = data.drop(['iata_code_x', 'iata_code_y','IATA_CODE_Reporting_Airline', 'Code'], axis=1)

# Remove rows missing data
data = data.dropna() 

### <font color='#8735fb'> Convert Categorical Data

The flight data contains several categorical columns (such as city names) that may be important features for our model. However, "Chicago" and "New York" are not in a format that RAPIDS can understand. We will use the built-in [`factorize`](https://docs.rapids.ai/api/cudf/stable/api.html?highlight=factorize#cudf.core.index.CategoricalIndex.factorize) function along with a few lines of Python to convert these strings or catgories into float32 values.

We retain the original `OriginCityName` column for human-readable data visualization and add a computer-readable `enc_OriginCityName` column.

We then finish up be setting `data` to be only the numeric columns we have just created, with the CityNames added back in.

In [7]:
# Encode - Convert human-readable names to corresponding computer-readable integers
encodings, mappings = data['OriginCityName'].factorize() # encode/categorize a sample feature 
numeric_columns = []
for colname in data.columns:
    if colname in categorical_columns:
        values = data[colname].astype('category').cat.codes.astype('float32')
        colname = 'enc_' + colname                
        data.insert(0, colname, values)                
    numeric_columns += [colname]  

In [8]:
# Remove redundant, surrogate, and unwanted columns from the data
remove_cols = set (['Year', 'Cancelled', 'DOT_ID_Reporting_Airline', 'enc_IATA_CODE_Reporting_Airline', 'ArrTime']); 
output_columns = list(set(numeric_columns).difference(remove_cols))

# Add back additional columns that are used for data visualization, but not training
output_columns = output_columns + ['OriginCityName', 'DestCityName']

processed_data = data[output_columns]

## <font color='#8735fb'> Putting It All Together

Now that we have done this all once, let's combine all the steps into a single function so that we can easily run it against any new data files.

We ran the previous cells againsts the smaller prototyping dataset. Now, let's run our combined function against the large dataset.

After processing the data, we will write it to a new .csv file. This allows us to use the processed data later for visualization and training, without the need to re-run the processing in this Notebook. 

In [9]:
# Save the smaller dataset to file
processed_data.to_csv(processed_data_file)
del(data); del(processed_data) # Delete the large DataFrames from memory

In [10]:
# Define a function to process an entire dataset 
import cudf

def process_data_file(data_file, carriers_file, cols, dtypes, categorical_columns):        
    # Ingest - Read the CSV files into the DataFrame
    data = cudf.read_csv(data_file, cols=cols, dtypes=dtypes)[cols] # Read in data, ignoring any column not in cols
    carriers = cudf.read_csv(carriers_file)
    airports = cudf.read_csv(airports_file, usecols=['iata_code', 'latitude_deg', 'longitude_deg',  'elevation_ft'])       
    
    # Merge - Combine the external data with the airline data         
    data = cudf.merge(data, carriers, left_on='IATA_CODE_Reporting_Airline', right_on='Code', how='left')
    data = cudf.merge(data, airports, left_on='Dest', right_on='iata_code', how='left')
    data = cudf.merge(data, airports, left_on='Origin', right_on='iata_code', how='left')            

    # Rename - Add clarity to the combined dataset by renaming columns
    data = data.rename(columns= { 'latitude_deg_x' : 'dest_lat', 'longitude_deg_x': 'dest_long',
                                  'latitude_deg_y' : 'origin_lat', 'longitude_deg_y': 'origin_long',
                                  'elevation_ft_x' : 'dest_elevation', 'elevation_ft_y' : 'origin_elevation',
                                  'Description' : 'Airline'})

    # Remove duplicates columns
    data = data.drop(['iata_code_x', 'iata_code_y','IATA_CODE_Reporting_Airline', 'Code'], axis=1)

    print(f'Added the following columns/features:\n { set(data.columns).difference(cols) }\n')
    print(f'Data currently has {data.shape[0]} rows and {data.shape[1]} columns\n')
    
    # Remove rows missing data
    data = data.dropna() 
    print(f'Dropping rows with missing or NA values, data now has {data.shape[0]} rows and {data.shape[1]} columns\n')
         

    # Encode - Convert human-readable names to corresponding computer-readable integers
    encodings, mappings = data['OriginCityName'].factorize() # encode/categorize a sample feature 
    print("Example encoding:")    
    numeric_columns = []
    for colname in data.columns:
        if colname in categorical_columns:
            values = data[colname].astype('category').cat.codes.astype('float32')
            colname = 'enc_' + colname                
            data.insert(0, colname, values)                
        numeric_columns += [colname]  
    print(list(zip(data['OriginCityName'][0:3].values_host, encodings[0:3])))

    # Remove redundant, surrogate, and unwanted columns from the data
    remove_cols = set (['Year', 'Cancelled', 'DOT_ID_Reporting_Airline', 'enc_IATA_CODE_Reporting_Airline', 'ArrTime']); 
    output_columns = list(set(numeric_columns).difference(remove_cols))

    # Add back additional columns that are used for data visualization, but not training
    output_columns = output_columns + ['OriginCityName', 'DestCityName']

    data = data[output_columns]
    print(f'Encoded and removed extra columns, data now has {data.shape[0]} rows and {data.shape[1]} columns\n')
    print(f'Removed: {remove_cols}')
    print(f'Returning: {output_columns}')

    return data

In [11]:
# Process the full dataset and save it to file
processed_data_full = process_data_file(data_file_full, carriers_file, cols, dtypes, categorical_columns)
processed_data_full.to_csv(processed_data_file_full)

Added the following columns/features:
 {'dest_lat', 'origin_elevation', 'dest_long', 'origin_lat', 'dest_elevation', 'origin_long', 'Airline'}

Data currently has 7422048 rows and 31 columns

Dropping rows with missing or NA values, data now has 7268232 rows and 31 columns

Example encoding:
[('Dallas/Fort Worth, TX', array(79, dtype=int16)), ('Dallas/Fort Worth, TX', array(79, dtype=int16)), ('Dallas/Fort Worth, TX', array(79, dtype=int16))]
Encoded and removed extra columns, data now has 7268232 rows and 30 columns

Removed: {'ArrTime', 'Cancelled', 'Year', 'DOT_ID_Reporting_Airline', 'enc_IATA_CODE_Reporting_Airline'}
Returning: ['dest_lat', 'origin_elevation', 'ArrDelay', 'enc_ArrTime', 'Quarter', 'enc_Flight_Number_Reporting_Airline', 'DayOfWeek', 'origin_lat', 'enc_Origin', 'DepDel15', 'enc_Dest', 'enc_DepTime', 'enc_CRSDepTime', 'DepDelay', 'AirTime', 'enc_OriginCityName', 'ArrDel15', 'Distance', 'dest_long', 'enc_DestCityName', 'OriginCityMarketID', 'enc_Airline', 'dest_elevati

# <font color='#8735fb'> Summary

In this Notebook, we learned how RAPIDS can be used to quickly merge and clean data files. We generated two prepocessed files that we will use for the remainder of the module.

* `airlines_processed_data_full.csv`: The full dataset, preproccessed
* `airlines_processed_data.csv`: A subset of the dataset, preprocessed

<center><a href="https://developer.nvidia.com/machine-learning"><img src="img/nvidia.png" alt="nvidia_logo" style="width: 1500px;"/></a></center>